In [10]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import logging
import joblib

# =====================================================
# Logging Setup
# =====================================================
LOG_FILE = "training_log.txt"
logging.basicConfig(
    filename=LOG_FILE,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def log(msg):
    print(msg)
    logging.info(msg)

# =====================================================
# Paths
# =====================================================
PROCESSED_DIR = "./Processed"
MODEL_SAVE_DIR = "./Models"
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# =====================================================
# Config
# =====================================================
SEQ_LEN = 8
BATCH_SIZE = 16
EPOCHS = 50
LR = 1e-4   # safer learning rate
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
log(f"🚀 Using device: {DEVICE.upper()}")

# =====================================================
# Dataset
# =====================================================
class StockDataset(Dataset):
    def __init__(self, df, seq_len=8):
        self.seq_len = seq_len
        # ✅ Removed Daily_Return
        self.features = df[["negative", "Open", "neutral", "positive", "Close", "EMA_7", "EMA_21"]].values
        self.targets = df["Close"].values
        self.samples = []
        for i in range(len(df) - seq_len - 1):
            x = self.features[i:i + seq_len]
            y = self.targets[i + seq_len]
            self.samples.append((x, y))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y = self.samples[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# =====================================================
# Model
# =====================================================
class StockLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # use last timestep
        out = self.fc(out)
        return out.squeeze(-1)

# =====================================================
# Training utilities
# =====================================================
def train_model(model, train_loader, val_loader, optimizer, criterion, epochs, company_name):
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for x, y in tqdm(train_loader, desc=f"Training {company_name} (Epoch {epoch+1}/{epochs})", leave=False):
            x, y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            preds = model(x)
            loss = criterion(preds, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # ✅ prevent exploding gradients
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        log(f"📊 [{company_name}] Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f}")

🚀 Using device: CUDA


In [11]:
# =====================================================
# ✅ Train each company using only *_train.csv
# =====================================================
for file in os.listdir(PROCESSED_DIR):
    if not file.endswith("_train.csv"):
        continue  # only train files

    company_path = os.path.join(PROCESSED_DIR, file)
    df = pd.read_csv(company_path)

    if df.shape[0] < SEQ_LEN + 2:
        log(f"⚠️ Skipping {file} (too few records)")
        continue

    company_name = file.replace("_train.csv", "")
    log(f"\n🏢 Training model for {company_name}")
    log(f"📄 Loaded {len(df)} training records")

    # =====================================================
    # Data Cleaning + Scaling
    # =====================================================
    df = df.sort_values("Date").reset_index(drop=True)
    df.columns = df.columns.str.strip()

    # Ensure all numeric columns are numeric
    for col in ["Close", "EMA_7", "EMA_21", "Open", "negative", "neutral", "positive"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Drop rows with missing price data
    df = df.dropna(subset=["Close", "EMA_7", "EMA_21", "Open"]).reset_index(drop=True)
    df[["negative", "neutral", "positive"]] = df[["negative", "neutral", "positive"]].fillna(0.0)

    # =====================================================
    # Scaling
    # =====================================================
    scaler = StandardScaler()
    df[["Close", "EMA_7", "EMA_21", "Open"]] = scaler.fit_transform(df[["Close", "EMA_7", "EMA_21", "Open"]])

    # ✅ Save fitted scaler for inference
    scaler_path = os.path.join(MODEL_SAVE_DIR, f"{company_name}_scaler.pkl")
    joblib.dump(scaler, scaler_path)
    log(f"🧠 Saved scaler for {company_name} at {scaler_path}")

    # =====================================================
    # Dataset + Dataloaders
    # =====================================================
    train_dataset = StockDataset(df, seq_len=SEQ_LEN)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    # =====================================================
    # Model, Optimizer, Loss
    # =====================================================
    torch.cuda.empty_cache()
    model = StockLSTM(input_dim=7).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.MSELoss()

    # =====================================================
    # Train Model
    # =====================================================
    train_model(model, train_loader, None, optimizer, criterion, EPOCHS, company_name)

    save_path = os.path.join(MODEL_SAVE_DIR, f"{company_name}_model.pt")
    torch.save(model.state_dict(), save_path)
    log(f"✅ Saved model to {save_path}")

log("\n🏁 Training complete for all companies!")


🏢 Training model for XOM_stock_gdelt_final
📄 Loaded 802 training records
🧠 Saved scaler for XOM_stock_gdelt_final at ./Models/XOM_stock_gdelt_final_scaler.pkl


📊 [XOM_stock_gdelt_final] Epoch 1/50 | Train Loss: 0.9022


📊 [XOM_stock_gdelt_final] Epoch 2/50 | Train Loss: 0.6699


📊 [XOM_stock_gdelt_final] Epoch 3/50 | Train Loss: 0.2315


📊 [XOM_stock_gdelt_final] Epoch 4/50 | Train Loss: 0.0368


...

📊 [AAPL_stock_gdelt_final] Epoch 46/50 | Train Loss: 0.2618


📊 [AAPL_stock_gdelt_final] Epoch 47/50 | Train Loss: 0.2600


📊 [AAPL_stock_gdelt_final] Epoch 48/50 | Train Loss: 0.2592


📊 [AAPL_stock_gdelt_final] Epoch 49/50 | Train Loss: 0.2566


📊 [AAPL_stock_gdelt_final] Epoch 50/50 | Train Loss: 0.2510
✅ Saved model to ./Models/AAPL_stock_gdelt_final_model.pt

🏁 Training complete for all companies!
